In [1]:
import json
import edge_validation as ev

with open('configuration/belfast.json') as f:
    config = json.load(f)

config

{'study_region': 'belfast',
 'to_crs': 'epsg:29902',
 'osm_graphml_path': 'data/Belfast/belfast_gb_2019_10000m_all_osm_20190902.graphml.xml',
 'osm_buffer_gpkg_path': 'data/Belfast/belfast_gb_2019_1600m_buffer.gpkg',
 'official_streets_shp_path': 'data/Belfast/Belfast_City_Council_Area_Street_Network.shp'}

In [2]:
%%time
# load gdf of edges from osm graph and the study area boundary polygon
gdf_osm_streets, gdf_official_streets, study_area = ev.load_data(config['osm_graphml_path'],
                                         config['osm_buffer_gpkg_path'],
                                         config['official_streets_shp_path'])
print(gdf_osm_streets.shape)
print(gdf_official_streets.shape)

(70312, 22)
(19757, 25)
CPU times: user 53.1 s, sys: 259 ms, total: 53.3 s
Wall time: 53.3 s


In [4]:
%%time
gdf_osm_streets_clipped, gdf_official_streets_clipped = ev.clip_data(gdf_osm_streets, gdf_official_streets, study_area)
print(gdf_osm_streets_clipped.shape)
print(gdf_official_streets_clipped.shape)

(26244, 22)
(18662, 25)
CPU times: user 27.1 s, sys: 3.93 ms, total: 27.1 s
Wall time: 27.1 s


In [ ]:
# Clip data so that all three datasets only include the study area as defined by the project
osm_data_clipped, official_clipped = ev.clip_data()

In [ ]:
# Plot the study area
ev.plot_map(gdf_study_area)

In [ ]:
# Plot the osm dataset edge network
ev.plot_map(gdf_osm)

In [ ]:
# Plot the official dataset edge network
ev.plot_map(gdf_official)

In [ ]:
# Plot the osm dataset edge network clipped to only include edges within study area
ev.plot_map(osm_data_clipped)

In [ ]:
# Plot the offical dataset edge network clipped to only include edges within study area
ev.plot_map(official_clipped)

In [ ]:
# Find the total length of the osm dataset edge network
ev.total_length(gdf_osm)

In [ ]:
# Find the total length of the official dataset edge network
ev.total_length(gdf_official)

In [ ]:
# Find the area of the official dataset edge network which intersects with the osm network at a specified buffer
# Change the buffer by replacing the integer in the first input
ev.buffer_intersected(5, official_buffer)

In [ ]:
# Find the area of the osm dataset edge network which intersects with the official network at a specified buffer
# Change the buffer by replacing the integer in the first input
# The buffer is measured in meters
ev.buffer_intersected(5, osm_buffer)

In [ ]:
# Find the percentage of overlap between the two networks at a specified buffer
# Change the buffer by replacing the integer in the first input
# The buffer is measured in meters
ev.shared_area(5)